## Import

In [11]:
from tkinter import * # __all__
from tkinter import filedialog
from time import gmtime, strftime, localtime

import xlwings as xw
import pandas as pd
import numpy as np
import os, re, flaml
import tkinter.ttk as ttk
import tkinter.messagebox as msgbox

## Functions

In [52]:
# 학습 데이터 파일 추가
def add_file():
    files = filedialog.askopenfilenames(title="데이터 선택", \
        filetypes=(("모든 파일", "*.*"),
                ("엑셀 파일", "*.xlsx"),
                ("csv 파일", "*.csv")),\
        #initialdir=r"C:\python")
        initialdir="./datasets/")
        # 최초에 사용자가 지정한 경로를 보여줌
    
    # 사용자가 선택한 파일 목록
    for idx, file in enumerate(files):
        list_file.insert(parent="", index = "end", iid = idx,text = file, value = (END, file))
        
# 선택 삭제
def del_file():
    selected_item = list_file.selection()[0]
    list_file.delete(selected_item)
    
    '''
    for index in reversed(list_file.curselection()):
        list_file.delete(index)
    '''

# 예측 데이터 파일 추가
def add_file2():
    files2 = filedialog.askopenfilenames(title="데이터 선택", \
        filetypes=(("모든 파일", "*.*"),
                ("엑셀 파일", "*.xlsx"),
                ("csv 파일", "*.csv")),\
        initialdir=r"C:\python")
        # 최초에 사용자가 지정한 경로를 보여줌
    
    # 사용자가 선택한 파일 목록
    for idx, file in enumerate(files2):
        list_file2.insert(parent="", index = "end", iid = idx,text = file, value = (END, file))
    
         
# 선택 삭제
def del_file2():
    selected_item = list_file2.selection()[0]
    list_file2.delete(selected_item)

    '''
    for index in reversed(list_file2.curselection()):
        list_file2.delete(index)
    '''

# 저장 경로 (폴더)
def browse_dest_path():
    folder_selected = filedialog.askdirectory()
    if folder_selected is None: # 사용자가 취소를 누를 때
        return
    txt_dest_path.delete(0, END)
    txt_dest_path.insert(0, folder_selected)

def read_xlsx(name):
    instance = xw.App(visible=False)
    xlsx_data = xw.Book(name).sheets[0]
    df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
    instance.quit()
    instance.kill()
    return df

def data_split(train_data, test_data, target_col, clf_opt = False):
    train_x = train_data.loc[:, ~train_data.columns.isin([target_col])]
    test_x = test_data.loc[:, ~test_data.columns.isin([target_col])]
    
    if clf_opt:
        train_y = np.array(train_data.loc[:, target_col], dtype = np.int32)
    else:
        train_y = np.array(train_data.loc[:, target_col])
    
    col_names = train_x.columns
    
    #return np.array(train_x), train_y, np.array(test_x), test_y, col_names
    return np.array(train_x), train_y, np.array(test_x), col_names

def callbackTarget(event):
    if ml_cate.get() == "분류 예측":
        # 타겟 컬럼명 설정
        target_space['state'] = 'normal'
        #ml_algo['values'] = ('전체', 'LightGBM', 'XGBoost', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression')
        ml_algo['values'] = ('전체', 'LightGBM', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression')
        
    if ml_cate.get() == "수치 예측":
        # 타겟 컬럼명 설정
        target_space['state'] = 'normal'
        #ml_algo['values'] = ('전체', 'LightGBM', 'XGBoost', 'CatBoost', 'RandomForest', 'SVR', 'ElasticNet', 'Linear Regression')
        ml_algo['values'] = ('전체', 'LightGBM', 'CatBoost', 'RandomForest', 'SVR', 'ElasticNet', 'Linear Regression')
        
    if ml_cate.get() == "시계열 예측":
        # 타겟 컬럼명 설정
        target_space['state'] = 'normal'
        
    elif ml_cate.get() == "군집 분석":
        # 타겟 컬럼명 설정
        target_space['state'] = 'disabled'        

def callbackSpin(event):
    if (ml_algo.get() in ['전체', 'SVM', 'Logistic Regression', 'SVR', 'ElasticNet', 'Linear Regression', 'RandomForest']) or (radio_var.get() == 2):
        cnt_spin.config(state=DISABLED)
    else:
        cnt_spin.config(state=NORMAL)

def radio_state():
    if (ml_algo.get() in ['전체', 'SVM', 'Logistic Regression', 'SVR', 'ElasticNet', 'Linear Regression', 'RandomForest']) or (radio_var.get() == 2):
        cnt_spin.config(state=DISABLED)
    else:
        cnt_spin.config(state=NORMAL)

In [ ]:
rebalancing_days = {"1M":30, "2M":60, "1Q": 90, "2Q": 180, "3Q": 270, "1Y":365}

* layout

In [105]:
root = Tk()
root.title("HUG-Portfolio Analysis")
root.option_add("*tearOff", False)

# Create a style
style = ttk.Style(root)

root.tk.call("source", "./add/forest-light.tcl")
style.theme_use("forest-light")
root.update_idletasks()

In [106]:
# 파일 프레임 (파일 추가, 선택 삭제)
file_frame = ttk.LabelFrame(root, text = "WINK 1171 or WINK 6002")
file_frame.pack(fill="x", padx=5, pady=5) # 간격 띄우기

btn_frame = Frame(file_frame)
btn_frame.pack(side = "right")

btn_add_file = ttk.Button(btn_frame, width=10,
                      text="파일추가", command=add_file, style = "Accent.TButton")
btn_add_file.grid(row = 1, padx = 5, pady = 1, sticky = "nsew")

btn_del_file = ttk.Button(btn_frame, width=10, 
                      text="선택삭제", command=del_file)
btn_del_file.grid(row = 2, padx = 5, pady = 1, sticky = "nsew")

In [107]:
# 리스트 프레임
list_frame = Frame(file_frame)
list_frame.pack(fill="both", padx=5, pady=5, side = "bottom")

scrollbar = ttk.Scrollbar(list_frame)
scrollbar.pack(side="right", fill="y")

list_file = ttk.Treeview(list_frame, selectmode = "extended", yscrollcommand=scrollbar.set, height = 1)
list_file.pack(side="left", fill="both", expand=True)
scrollbar.config(command=list_file.yview)

list_file.column('#0')
list_file.heading('#0', text = "Data List", anchor = "center")

{}

In [108]:
# 옵션 프레임
frame_option = ttk.Frame(root)
frame_option.pack(padx=5, pady=5, ipady=5)

notebook = ttk.Notebook(frame_option)
ml_options = ttk.Frame(notebook)

ml_top = ttk.Frame(ml_options)
ml_bottom = ttk.Frame(ml_options)

ml_top.pack(side = "left")
ml_bottom.pack(side = "right")

ml_cate_label = Label(ml_top, text="Rebalancing Cycles", width=18)
ml_cate_label.grid(row = 0, column=0, padx=2, pady=5)

In [109]:
# 1. rebalancing settings
opt_ml_cate = ['1M', '2M', '1Q', '2Q', '3Q', '1Y']
ml_cate = ttk.Combobox(ml_top, state="readonly", values=opt_ml_cate, width=15)
ml_cate.current(0)
ml_cate.bind("<<ComboboxSelected>>", callbackTarget)
ml_cate.grid(row = 0, column=1, padx=2, pady=5)

# 2. optimization methods 선택
ml_algo_label = Label(ml_bottom, text="예측 모델", width=12)
ml_algo_label.grid(row = 0, column=2, padx=2, pady=5)

opt_ml_algo = ['전체', 'LightGBM', 'CatBoost', 'RandomForest', 'SVM', 'Logistic Regression']
ml_algo = ttk.Combobox(ml_bottom, state="readonly", values=opt_ml_algo, width=15)
ml_algo.current(0)
ml_algo.bind("<<ComboboxSelected>>", callbackSpin)
ml_algo.grid(row = 0, column=3, padx=2, pady=5)

# 3. 하이퍼파라미터 튜닝 선택 옵션
radio_var = IntVar(value = 2)
radio_frame = ttk.Frame(ml_top)

ml_hpo_label = Label(ml_top, text="모델 튜닝 여부", width=15)
ml_hpo_label.grid(row = 1, column=0, padx=2, pady=5)

hpo_y = ttk.Radiobutton(radio_frame, text ="Y", variable = radio_var,
                        value = 1, command = radio_state)
hpo_y.pack(side="left", padx=2, pady=5)

hpo_n = ttk.Radiobutton(radio_frame, text ="N", variable = radio_var,
                        value = 2, command = radio_state)
hpo_n.pack(side="left", padx=2, pady=5)

radio_frame.grid(row=1, column=1,padx=2,pady=5)

# iteration 숫자
hpo_cnt = Label(ml_bottom, text="튜닝 횟수", width=15)
hpo_cnt.grid(row = 1, column=2, padx=2, pady=5)

current_value = IntVar(value = 0)

cnt_spin = ttk.Spinbox(ml_bottom, from_=0, to=500, width = 11,
                       textvariable = current_value)
cnt_spin.grid(row = 1, column=3, padx=2, pady=5)

notebook.add(ml_options, text = "Portfolio Settings")

column_options = ttk.Frame(frame_option)
column_top = ttk.Frame(column_options)
column_bottom = ttk.Frame(column_options)

column_top.grid(row=0)
column_bottom.grid(row=1)

# 4. 타겟 컬럼 처리 옵션
target_format = Label(column_top, text="타겟열 이름", width=12)
target_format.grid(row=0, column =0, padx=2, pady=5)

target_space = ttk.Entry(column_top,width = 20)
target_space.grid(row=0, column=1, padx=2, pady=5)

# 5. 기준 컬럼 처리 옵션
index_format = Label(column_top, text="기준열 이름", width=12)
index_format.grid(row=0, column=2, padx=5, pady=5)

index_space = ttk.Entry(column_top, width = 20)
index_space.grid(row=0, column=3, padx=2, pady=5)

# 6. 제거 컬럼 처리 옵션
except_format = Label(column_bottom, text="제거열 이름", width=12)
except_format.grid(row=0, column=0, padx=2, pady=5)

except_space = ttk.Entry(column_bottom, width = 59)
except_space.grid(row=0, column=1, padx=2, pady=5)

notebook.add(column_options, text = "Column Options")
notebook.pack()

# 실행 프레임
frame_run = Frame(root)
frame_run.pack(fill="x", padx=5, pady=5)

btn_close = ttk.Button(frame_run, text="닫기", width=12, command=root.quit)
btn_close.pack(side="right", padx=5, pady=5)

btn_start = ttk.Button(frame_run, text="시작", width=12,
                  command = main_functions)
btn_start.pack(side="right", padx=5, pady=5)

In [110]:
root.resizable(True, True)
root.mainloop()

In [4]:
# 파일 프레임 (파일 추가, 선택 삭제)
file_frame2 = ttk.LabelFrame(root, text = "예측 데이터")
file_frame2.pack(fill="x", padx=5, pady=5) # 간격 띄우기

btn_frame2 = Frame(file_frame2)
btn_frame2.pack(side = "right")

btn_add_file2 = ttk.Button(btn_frame2, width=10,
                       text="파일추가", command=add_file2, style = "Accent.TButton")
btn_add_file2.grid(row = 1, padx = 5, pady = 1, sticky = "nsew")

btn_del_file2 = ttk.Button(btn_frame2, width=10,
                       text="선택삭제", command=del_file2)
btn_del_file2.grid(row = 2, padx = 5, pady = 1, sticky = "nsew")

# 리스트 프레임
list_frame2 = Frame(file_frame2)
list_frame2.pack(fill="both", padx=5, pady=5)

scrollbar2 = ttk.Scrollbar(list_frame2)
scrollbar2.pack(side="right", fill="y")

list_file2 = ttk.Treeview(list_frame2, selectmode = "extended", yscrollcommand=scrollbar2.set, height = 1)
list_file2.pack(side="left", fill="both", expand=True)
scrollbar2.config(command=list_file2.yview)

list_file2.column('#0')
list_file2.heading('#0', text = "Data List", anchor = "center")

# 저장 경로 프레임
path_frame = ttk.LabelFrame(root, text="저장경로")
path_frame.pack(fill="x", padx=5, pady=5, ipady=5)

txt_dest_path = ttk.Entry(path_frame)
txt_dest_path.pack(side="left", fill="x", expand=True, padx=5, pady=5, ipady=4) # 높이 변경

btn_dest_path = ttk.Button(path_frame, text="찾아보기", width=10,
                      command = browse_dest_path, style = "Accent.TButton")
btn_dest_path.pack(side="right", padx=5, pady=5)



root.resizable(True, True)
root.mainloop()